
<div style="color:#ffffff;
          font-size:50px;
          font-style:italic;
          text-align:left;
          font-family: 'Lucida Bright';
          background:#4686C8;">
  	&nbsp; Q&A with Fine-Tuning
</div>
<br>   
<div style="
          font-size:20px;
          text-align:left;
          font-family: 'Palatino';
          ">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Project: Q&A using pretrained model with Fine-Tuning<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Author: George Barrinuevo<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Date: 07/09/2025<br>
</div>

<br><div style="color:#ffffff;
          font-size:30px;
          font-style:italic;
          text-align:left;
          font-family: 'Lucida Bright';
          background:#4686C8;">
  	      &nbsp; Project Notes
</div>
<div style="
          font-size:16px;
          text-align:left;
          font-family: 'Cambria';">
    
<b>My Thoughts</b><br>
- This script demonstrates creating a model specifically for Question and Answering tasks. This is for educational purposes only.
- A pretrained model that is also already fine-tuned for Question and Answering task is used. Using pretrained models saves a lot of cost and time compared with training the entire model.
- The notebook was developed on Google Colab.

<b>Technical Details</b><br>
<u>Tokenizer for model</u>
- The tokenizer is used to data pre-process the text corpus data to a format the model can use. It is highly recommended to use the tokenizer specific to the model being used.

<u>Model</u>
- A pretrained model is used to save cost and time in training a large model. Later, fine-tuning methods are used to make the model work on specific tasks.
- The pretrained model bert-large-uncased-whole-word-masking-finetuned-squad is already fine-tuned to the SQuAD 1.1 dataset, which uses the BertForQuestionAnswering architecture. This architecture specializes in producing an answer based on a question and text.

<u>Dataset</u>
- SQuAD (Stanford Question Answering Dataset) is a dataset designed for training and evaluating question answering systems. This dataset is download and saved, so it can be reloaded for later use.

<u>NLP parser</u>
- Spacy is in NLP tokenizer parser. From the input text, it will create the tokenizer, tagger, parser and NER (named entity recogniztion). See https://spacy.io/.

<u>Lemmatization</u>
- Lemmatization is grouping the various forms of a word. An example is 'walks', 'walking' and 'walked' are part of the base word 'walk'. The 'walk' version is it's lemma.

<u>Tokenizing the corpus text</u>
- For tokenizing the corpus text (dataset) TF–IDF (term frequency–inverse document frequency) is used. The 'bag of words' disregards word order (and thus most of syntax or grammar), but uses term frequency. An improvement on this is TF-IDF where importance of a word is determined by using 'term frequency'. But for common words uses 'inverse document frequency' where it reduces it's weight if it appears in multiple documents.

<u>pipeline process</u>
- All of the above is assembled in a pipeline:<br>
      SQuAD -> spacy -> lemma -> TF-IDF -> tokenizer -> model -> answer


In [1]:
# from huggingface_hub import notebook_login
# notebook_login()


In [2]:
!pip install torch transformers sklearn spacy[cuda92]

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import os
import requests
import random
import pickle
import pandas as pd
import json
import spacy
import numpy as np
import torch

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, BertTokenizer, BertForQuestionAnswering
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
CACHE_DIR = os.getenv('QA_CACHE_PATH', 'cache')
DATA_DIR = os.getenv('QA_DATA_PATH', 'data')

SQUAD_URL = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
SQUAD_TRAIN = f"{DATA_DIR}/train-v2.0.json"
LEMMA_CACHE = f"{CACHE_DIR}/lemmas.feather"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TOPK = 10 if DEVICE.type == 'cuda' else 5

In [5]:
# os.system("jupyter nbextension enable --py widgetsnbextension")
#     The above line is commented out since it causes rendering issues in Github.
os.system("python3 -m spacy download en_core_web_sm")

if not os.path.isdir(DATA_DIR):
    os.mkdir(DATA_DIR)
if not os.path.isdir(CACHE_DIR):
    os.mkdir(CACHE_DIR)

print("Downloading pretrained models to cache")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', return_token_type_ids=False)
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

if not os.path.isfile(SQUAD_TRAIN):
    print(f"Downloading squad dataset as {SQUAD_TRAIN}")
    response = requests.get(SQUAD_URL, stream=True)

    print(f'Saving SQUAD data')
    with open(SQUAD_TRAIN, "wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)

print(f'All done')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Saving SQUAD data


42123633it [04:09, 169037.69it/s]

All done


In [6]:
spacy.prefer_gpu()
sp = spacy.load("en_core_web_sm")

In [7]:
# If there is any issues with loading the below file, than delete the SQUAD_TRAIN directory and re-run.
# this notebook. The notebook will download a fresh copy of the file.
# This code will load the saved SQuAD data.

with open(SQUAD_TRAIN) as f:
    doc = json.load(f)
doc.keys(), type(doc["data"]), len(doc["data"])

(dict_keys(['version', 'data']), list, 442)

In [8]:
# Extracts the paragraphs (context) and questions from the SQuAD dataset.
# If there is any issues with loading the below file, than delete the SQUAD_TRAIN directory and re-run this notebook.
# The notebook will download a fresh copy of the file and save it for reloading for next script run.

paragraphs = []
questions = []
for topic in doc["data"]:
    for pgraph in topic["paragraphs"]:
        paragraphs.append(pgraph["context"])
        for qa in pgraph["qas"]:
            if not qa["is_impossible"]:
                questions.append(qa["question"])

len(paragraphs), len(questions), random.sample(paragraphs, 2), random.sample(questions, 5)

(19035,
 86821,
 ["Comcast Corporation, formerly registered as Comcast Holdings,[note 1] is an American multinational mass media company and is the largest broadcasting and largest cable company in the world by revenue. It is the second largest pay-TV company after the AT&T-DirecTV acquisition, largest cable TV company and largest home Internet service provider in the United States, and the nation's third largest home telephone service provider. Comcast services U.S. residential and commercial customers in 40 states and the District of Columbia. The company's headquarters are located in Philadelphia, Pennsylvania.",
  'Most modern air defence systems are fairly mobile. Even the larger systems tend to be mounted on trailers and are designed to be fairly quickly broken down or set up. In the past, this was not always the case. Early missile systems were cumbersome and required much infrastructure; many could not be moved at all. With the diversification of air defence there has been much

In [9]:
def lemmatize(phrase):
    return " ".join([word.lemma_ for word in sp(phrase)])

In [10]:
# Load the lemmatization cache. If it does not exist, then create it by lemmatization of every paragraph.
# The 'lemmas' list will contain the lemmatization version of the corresponding 'paragraphs'.
# The tqdm() is used to display the progress bar.

%%time

if not os.path.isfile(LEMMA_CACHE):
    lemmas = [lemmatize(par) for par in tqdm(paragraphs)]
    df = pd.DataFrame(data={'context': paragraphs, 'lemmas': lemmas})
    df.to_feather(LEMMA_CACHE)

df = pd.read_feather(LEMMA_CACHE)
paragraphs = df.context
lemmas = df.lemmas

100%|██████████| 19035/19035 [12:04<00:00, 26.28it/s]


CPU times: user 11min 15s, sys: 3.87 s, total: 11min 18s
Wall time: 12min 4s


In [11]:
# Print out an example of a paragraph and lemma.
print(f'A paragraph: {paragraphs[1]}')
print(f'A lemma: {lemmas[1]}')


A paragraph: Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and 

In [12]:
# The code below will load the saved TF-IDF data, but if it does not exist will create this data and save it for later use.
# A stop-word are words that have little semantic value and are therefore filtered out.
# The code will create a TF-IDF from the Lemmatization of the paragraphs (context) and save this data to be loaded the next time.

%%time
VECTOR_CACHE = 'cache/vectors.pickle'
if not os.path.isfile(VECTOR_CACHE):
    vectorizer = TfidfVectorizer(
        stop_words='english', min_df=5, max_df=.5, ngram_range=(1,3))
    tfidf = vectorizer.fit_transform(lemmas)
    with open(VECTOR_CACHE, "wb") as f:
        pickle.dump(dict(vectorizer=vectorizer, tfidf=tfidf), f)
else:
    with open(VECTOR_CACHE, "rb") as f:
        cache = pickle.load(f)
        tfidf = cache["tfidf"]
        vectorizer = cache["vectorizer"]

len(vectorizer.vocabulary_)

CPU times: user 8.9 s, sys: 1.52 s, total: 10.4 s
Wall time: 10.6 s


34552

In [13]:
# This code will create a Lemmatization on the question then converts it to TF-IDF weights.

question = "When did the last country to adopt the Gregorian calendar start using it?"
t_lemmatize = lemmatize(question)
query = vectorizer.transform([t_lemmatize])
(query > 0).sum(), vectorizer.inverse_transform(query)
print(f't_lemmatize: {t_lemmatize}')
print(f'query: {query}')

t_lemmatize: when do the last country to adopt the gregorian calendar start use it ?
query: <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 34552)>
  Coords	Values
  (0, 2372)	0.24788836762683059
  (0, 5792)	0.34415666630537717
  (0, 8571)	0.17385535976562153
  (0, 8573)	0.4636364467412126
  (0, 14446)	0.3848259593889299
  (0, 14447)	0.4167582282940414
  (0, 29795)	0.20985246114065498
  (0, 29812)	0.43750217041787265
  (0, 32566)	0.1330355626488098


In [14]:
# This code will calculate the score for query and paragraph matches and sorts them. The 3 highest scoring
# paragraphs (context) are displayed.

%%time
scores = (tfidf * query.T).toarray()
results = (np.flip(np.argsort(scores, axis=0)))
[paragraphs[i] for i in results[:3, 0]]
# paragraphs[results[0]]

CPU times: user 9.59 ms, sys: 9 µs, total: 9.6 ms
Wall time: 9.68 ms


['During the period between 1582, when the first countries adopted the Gregorian calendar, and 1923, when the last European country adopted it, it was often necessary to indicate the date of some event in both the Julian calendar and in the Gregorian calendar, for example, "10/21 February 1750/51", where the dual year accounts for some countries already beginning their numbered year on 1 January while others were still using some other date. Even before 1582, the year sometimes had to be double dated because of the different beginnings of the year in various countries. Woolley, writing in his biography of John Dee (1527–1608/9), notes that immediately after 1582 English letter writers "customarily" used "two dates" on their letters, one OS and one NS.',
 "In conjunction with the system of months there is a system of weeks. A physical or electronic calendar provides conversion from a given date to the weekday, and shows multiple dates for a given weekday and month. Calculating the day o

In [15]:
# Filter for the first 10 highest scoring query to paragraph matches that pass a threshold value and put in a dataframe
# format broken down to 'question' and 'context' sections. Store results in a cache file.

%%time
THRESH = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:10, 0] ]
contexts = [ (paragraphs[i],s)
    for (i,s) in candidate_idxs if s > THRESH ]

question_df = pd.DataFrame.from_records([ {
    'question': question,
    'context':  ctx
} for (ctx,s) in contexts ])

question_df.to_feather("cache/question_context.feather")

CPU times: user 3.62 ms, sys: 1.98 ms, total: 5.59 ms
Wall time: 6.3 ms


In [16]:
# This implements the Question & Answer Fine-Tunning model using the above data.

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

print(f'-----------------')
t_record = question_df.to_dict(orient="records")[9]
question = t_record['question']
text = t_record['context']
print(f'\nquestion: {question}')
print(f'\ntext: {text}')

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits)

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
t_answer = tokenizer.decode(predict_answer_tokens)
print(f'\nanswer: {t_answer}')

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

-----------------

question: When did the last country to adopt the Gregorian calendar start using it?

text: Philip II of Spain decreed the change from the Julian to the Gregorian calendar, which affected much of Roman Catholic Europe, as Philip was at the time ruler over Spain and Portugal as well as much of Italy. In these territories, as well as in the Polish–Lithuanian Commonwealth (ruled by Anna Jagiellon) and in the Papal States, the new calendar was implemented on the date specified by the bull, with Julian Thursday, 4 October 1582, being followed by Gregorian Friday, 15 October 1582. The Spanish and Portuguese colonies followed somewhat later de facto because of delay in communication.

answer: 15 october 1582
